In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import pandas as pd
import random
import math

In [ ]:
class Neural_Network():
    def __init__(self, NN_structure, alpha, max_iter):
        self.NN_structure = NN_structure
        self.n_layers = len(NN_structure)
        self.alpha = alpha
        self.max_iter = max_iter
    
    # Defind some types of activation functions and its derivatives:
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def d_sigmoid(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def linear(self,x):
        return x
    
    def d_linear(self,x):
        return 1

    def relu(self,x):
        return np.maximum(0,x)

    def d_relu(self,x):
        dx = np.where(x <= 0, 0, 1)
        return dx

    def tanh(self,x):
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def d_tanh(self,x):
        return 1 - self.tanh(x)**2
    
    
    # Initialize weights:
    def init_weights(self, seed):
        np.random.seed(seed)
        self.theta_ = {}
        for L in range(1, self.n_layers):
#             self.theta_['Layer' + str(L+1)] = np.ones((self.NN_structure[L]['n_neurons'], \
#                                                         self.NN_structure[L-1]['n_neurons'] + 1)) * 0.2
            self.theta_['Layer' + str(L+1)] = np.random.randn(self.NN_structure[L]['n_neurons'], \
                                                        self.NN_structure[L-1]['n_neurons'] + 1) * 0.1
        return self.theta_
    

    def add_bias(self, Xi):
        Xi = np.concatenate(([1], Xi))
        return Xi
    
    def copy_column(self, Xi, n):
        Ai = Xi.T
        for _ in range(n-1):
            Ai = np.concatenate((Ai, Xi.T), axis = 1)
        return Ai
    
    def forward_propagation(self, Xi):
        self.z_ = {}
        self.a_ = {}
        self.z_['2'] = np.dot(self.theta_['Layer2'], self.add_bias(Xi).T)
        self.a_['2'] = eval('self.'+self.NN_structure[1]['activ_func'])(self.z_['2'])
        
        for L in range (2, self.n_layers):
            self.a_[str(L)] = self.add_bias(self.a_[str(L)])
            self.z_[str(L+1)] = np.dot(self.theta_['Layer'+str(L+1)],self.a_[str(L)])
            self.a_[str(L+1)] = eval('self.'+self.NN_structure[L]['activ_func'])(self.z_[str(L+1)])
        return self
            
    def fit(self, X, y):    # including training and validation
        self.init_weights(seed = 10)
        self.costs = []
        self.R_sq_record = []
        n_train = math.ceil(0.9 * X.shape[0])
        n_validate = X.shape[0] - n_train
        
        X_train = X[0 : n_train]
        y_train = y[0 : n_train]
        X_validate = X[n_train : X.shape[0]]
        y_validate = y[n_train : X.shape[0]]
        
        self.n_iter = 0
        for _ in range(self.max_iter):
            cost = 0
            for i in range(X_train.shape[0]):
                self.delta = {}
                self.update = {}

                self.forward_propagation(X_train[i])
                self.delta[str(self.n_layers)] = self.a_[str(self.n_layers)] - y_train[i]
                delta_copy = self.copy_column(np.array([self.delta[str(self.n_layers)]]), \
                                              self.NN_structure[self.n_layers-2]['n_neurons'] + 1)

                self.update[str(self.n_layers)] = np.multiply(self.a_[str(self.n_layers-1)], delta_copy)

                cost += ((self.a_[str(self.n_layers)]-y_train[i])**2/2/X_train.shape[0]).sum()
                
                self.a_['1'] = self.add_bias(X_train[i])
                for L in range(self.n_layers-1, 1, -1):
                    theta_back = np.delete(self.theta_['Layer'+str(L+1)], 0, 1)
                    self.delta[str(L)] = np.multiply(np.dot(theta_back.T, self.delta[str(L+1)]), \
                        eval('self.d_' + self.NN_structure[L-1]['activ_func'])(self.z_[str(L)]))
                    delta_copy = self.copy_column(np.array([self.delta[str(L)]]), \
                                              self.NN_structure[L-2]['n_neurons'] + 1)
                    self.update[str(L)] = np.multiply(self.a_[str(L-1)], delta_copy)

                for L in range(2, self.n_layers+1):
                    self.theta_['Layer'+str(L)] -= self.alpha * self.update[str(L)]
                        
            
            sum_theta_sq = 0
            for L in range(2, self.n_layers + 1):
                sum_theta_sq += np.square(self.theta_['Layer'+str(L)]).sum()
            
            # Validation step:
            SS_res = 0
            for j in range(X_validate.shape[0]):
                self.forward_propagation(X_validate[j])
                error_each_sample = ((self.a_[str(self.n_layers)] - y_validate[j])**2).sum()
                SS_res += error_each_sample
#             y_bar = y_validate.mean(0)
#             SS_total = ((y_validate - y_bar)**2).sum()
            SS_total = (y_validate**2).sum()
            R_sq = (1 - SS_res/ SS_total)
            if R_sq >= 0.9:
                break
            self.n_iter += 1
            self.R_sq_record.append(R_sq)
            self.costs.append(cost)
        return self

In [ ]:
input_data = pd.read_excel('Input_AHM.xlsx')
target = pd.read_excel('Output_AHM.xlsx')
field_data = pd.read_excel('Field_data.xlsx')
y_merge = np.concatenate((target, field_data), axis = 0)
y_merge = np.log(y_merge)

np.random.seed(101)
input_data, target = shuffle(input_data, target)

n_train = math.ceil(0.90 * input_data.shape[0])           # 90% data for training and validation
n_test = input_data.shape[0] - n_train                    # the rest for blind testing

X = input_data.values
y = target.values
y = np.log(y)
field_data = field_data.values

def normalize(X):
    A = X.copy()
    mean = X.mean(axis = 0)
    diff = X.max(axis = 0) - X.min(axis = 0)
    for j in range(0, X.shape[1]):  
        A[:,j] = (X[:,j] - mean[j]) / diff[j]
    return A

def denormalize(X_norm, mean_X, diff_X):
    X_denorm = X_norm.copy()
    for j in range(X_norm.shape[0]):  
        X_denorm[j] = X_norm[j] * diff_X[j] + mean_X[j]
    return X_denorm

mean_X = X.mean(axis = 0)
mean_y = y.mean(axis = 0)
diff_X = X.max(axis = 0) - X.min(axis = 0)
diff_y = y.max(axis = 0) - y.min(axis = 0)

# Normalization:
X_norm = normalize(X)
y_norm = normalize(y)

# Define ranges of parameters:
bound_min = X_norm.min(axis = 0)
bound_max = X_norm.max(axis = 0)
diff = np.fabs(bound_max - bound_min)
bounds = []
for i in range(bound_min.shape[0]):
    bounds.append((bound_min[i], bound_max[i]))

# Normalize the field oil production:
y_merge_norm = normalize(y_merge)
field_data_norm = y_merge_norm[-1]

# Split the dataset:
X_train = X_norm[0 : n_train]
y_train = y_norm[0 : n_train]
X_test = X_norm[n_train : input_data.shape[0]]
y_test = y_norm[n_train : input_data.shape[0]]

In [ ]:
# Test case 0: BEST case
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},   
                {'n_neurons' : 45, 'activ_func' : 'tanh'},
                {'n_neurons' : 80, 'activ_func' : 'tanh'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.008, max_iter = 500) 

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 2), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = (np.exp(y_test_denorm)**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()

In [ ]:
# Test case 1:
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},   
                {'n_neurons' : 40, 'activ_func' : 'sigmoid'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.008, max_iter = 500)

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = ((np.exp(y_test_denorm))**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()

In [ ]:
# Test case 2: (second best)
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},   
                {'n_neurons' : 45, 'activ_func' : 'tanh'},
                {'n_neurons' : 80, 'activ_func' : 'tanh'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.01, max_iter = 500)

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 2), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = ((np.exp(y_test_denorm))**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()

In [ ]:
# Test case 3: (third best)
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},
                {'n_neurons' : 100, 'activ_func' : 'relu'},
                {'n_neurons' : 120, 'activ_func' : 'relu'},
                {'n_neurons' : 160, 'activ_func' : 'relu'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.01, max_iter = 500)

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 2), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = ((np.exp(y_test_denorm))**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()

In [ ]:
# Test case 4:
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},   
                {'n_neurons' : 90, 'activ_func' : 'sigmoid'},
                {'n_neurons' : 150, 'activ_func' : 'relu'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.01, max_iter = 500)

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 2), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = ((np.exp(y_test_denorm))**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()

In [ ]:
# Test case 5: a good case
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},   
                {'n_neurons' : 90, 'activ_func' : 'sigmoid'},
                {'n_neurons' : 180, 'activ_func' : 'relu'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.009, max_iter = 500)

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = ((np.exp(y_test_denorm))**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()

In [ ]:
# Test case 7: a good case
NN_structure = [{'n_neurons' : 15, 'activ_func' : 'none'},   
                {'n_neurons' : 45, 'activ_func' : 'tanh'},
                {'n_neurons' : 80, 'activ_func' : 'sigmoid'},
                {'n_neurons' : 180, 'activ_func' : 'relu'},
                {'n_neurons' : 50, 'activ_func' : 'linear'}]       

ANN = Neural_Network(NN_structure, alpha = 0.008, max_iter = 500) 

ANN.fit(X_train, y_train)

if ANN.n_iter < ANN.max_iter:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()
else:
    plt.plot(range(1, ANN.n_iter + 1), ANN.costs)
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost value')
    plt.show()

plt.plot(range(1, ANN.n_iter + 1), ANN.R_sq_record)
plt.xlabel('Number of iterations')
plt.ylabel('R squared value')
plt.show()

# Blind testing:
SS_res = 0
y_predict = np.array([])
for i in range(n_test):
    ANN.forward_propagation(X_test[i])
    y_predict = np.concatenate((y_predict, ANN.a_[str(ANN.n_layers)]), axis = 0)
    error_each_sample = ((ANN.a_[str(ANN.n_layers)] - y_test[i])**2).sum()
    SS_res += error_each_sample
y_predict = y_predict.reshape(y_test.shape)

y_bar = y_test.mean(0)
print(SS_res)
SS_total = ((y_test - y_bar)**2).sum()
# SS_total = (y_test**2).sum()
R_sq = (1 - abs(SS_res)/ SS_total)
print(R_sq)

for i in range(y_test.shape[0]):
    y_test_denorm = denormalize(y_test[i, :], mean_y, diff_y)
    y_predict_denorm = denormalize(y_predict[i, :], mean_y, diff_y)
    y_test_bar = y_test_denorm.mean(0)
    SS_res_1 = ((np.exp(y_test_denorm) - np.exp(y_predict_denorm))**2).sum()
    SS_total_1 = ((np.exp(y_test_denorm))**2).sum()
    R_sq_1 = (1 - abs(SS_res_1)/ SS_total_1)
    print('Accuracy = ', R_sq_1)
    plt.plot(range(1, 51), np.exp(y_test_denorm))
    plt.plot(range(1, 51), np.exp(y_predict_denorm))
    plt.legend(['Target','Predict'])
    plt.show()